<a href="https://colab.research.google.com/github/varamba/Ya_practicum_projects/blob/main/project_final_sales_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Дашборд для проекта: https://public.tableau.com/views/Final_project_Cosmo-Brothers/Dashboard2?:language=en-US&:display_count=n&:origin=viz_share_link

## Введение
Поступила задача от отдела маркетинга игры "Космическе братья"
Нужно сформировать модель монетизации игрового приложения.
Многие игры зарабатывают с помощью рекламы. И все они сталкиваются с противоречием;
- Пользователь разозлится и уйдёт, если начать показывать ему рекламу раньше, чем игра его затянет.
- Но  чем позже создатели игры включат рекламу, тем меньше они заработают.

Требуется помочь бизнесу выбрать оптимальное время для запуска рекламы. Зная расходы на продвижение игры, можно рассчитать её окупаемость при разных сценариях;
Пока создатели игры планируют показывать её на экране выбором постройки. Надо помочь не уйти в минус.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math as mth
from scipy import stats as st
import datetime as dt
from pandas.plotting import register_matplotlib_converters
from plotly import graph_objects as go
import plotly.express as px
import re
import warnings 
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
costs, sources, actions = (
    pd.read_csv('/dfs/ad_costs.csv', sep=',', engine = 'python'),
    pd.read_csv('/dfs/user_source.csv', sep=',', engine = 'python'),
    pd.read_csv('/dfs/game_actions.csv', sep=',', engine = 'python'))

## Предобработка
- Найти пропуски и дупликаты, изменить типы данных, если нужно.
- Добавить столбец по типу уровень завершен - да/нет.

In [4]:
tabs = [costs, actions, sources]
for t in tabs:
    display(t.head())
    display(t.info())
    display(t.isna().mean())
    display(t.duplicated().sum())

,source,day,cost
0,facebook_ads,2020-05-03,935.882786
1,facebook_ads,2020-05-04,548.354480
2,facebook_ads,2020-05-05,260.185754
3,facebook_ads,2020-05-06,177.982200
4,facebook_ads,2020-05-07,111.766796


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   source  28 non-null     object 
 1   day     28 non-null     object 
 2   cost    28 non-null     float64
dtypes: float64(1), object(2)
memory usage: 800.0+ bytes


None

source    0.0
day       0.0
cost      0.0
dtype: float64

0

...

,user_id,source
0,0001f83c-c6ac-4621-b7f0-8a28b283ac30,facebook_ads
1,00151b4f-ba38-44a8-a650-d7cf130a0105,yandex_direct
2,001aaea6-3d14-43f1-8ca8-7f48820f17aa,youtube_channel_reklama
3,001d39dc-366c-4021-9604-6a3b9ff01e25,instagram_new_adverts
4,002f508f-67b6-479f-814b-b05f00d4e995,facebook_ads


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13576 entries, 0 to 13575
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  13576 non-null  object
 1   source   13576 non-null  object
dtypes: object(2)
memory usage: 212.2+ KB


None

user_id    0.0
source     0.0
dtype: float64

0

Изменяю тип дат.

In [5]:
actions.rename(columns={'event_datetime' : 'dt'}, inplace=True)
actions['dt'] = pd.to_datetime(actions['dt'])
costs['day'] = pd.to_datetime(costs['day']).dt.date

Показываю дупликат

In [6]:
dups = actions[actions.duplicated(['user_id', 'dt'])]
dups

,dt,event,building_type,user_id,project_type
74891,2020-05-10 18:41:56,building,research_center,c9af55d2-b0ae-4bb4-b3d5-f32aa9ac03af,NaN


In [7]:
actions = actions.drop_duplicates().reset_index(drop=True)

Проверяю значения в столбцах, чтобы сравнить с ТЗ.

In [8]:
display(actions['event'].unique().tolist())
display(actions['building_type'].unique().tolist())
display(actions['project_type'].unique().tolist())

['building', 'finished_stage_1', 'project']

['assembly_shop', 'spaceport', nan, 'research_center']

[nan, 'satellite_orbital_assembly']

Добавляю столбец с завершением уровня.

In [9]:
actions['lvl_finished'] = actions.groupby('user_id')['event'].transform(lambda x: x.eq('finished_stage_1').any())
actions.head()

,dt,event,building_type,user_id,project_type,lvl_finished
0,2020-05-04 00:00:01,building,assembly_shop,55e92310-cb8e-4754-b622-597e124b03de,NaN,False
1,2020-05-04 00:00:03,building,assembly_shop,c07b1c10-f477-44dc-81dc-ec82254b1347,NaN,True
2,2020-05-04 00:00:16,building,assembly_shop,6edd42cc-e753-4ff6-a947-2107cd560710,NaN,True
3,2020-05-04 00:00:16,building,assembly_shop,92c69003-d60a-444a-827f-8cc51bf6bf4c,NaN,False
4,2020-05-04 00:00:35,building,assembly_shop,cdc6bb92-0ccb-4490-9866-ef142f09139d,NaN,True


Считаю количество пользователей, завершивших уровень.

In [10]:
actions['lvl_finished'].value_counts()

True     68469
False    67170
Name: lvl_finished, dtype: int64

Добавлю столбец с ценой 1 просмотра в меню строительства.

In [11]:
conditions = [
    (actions['event'] == 'building'),
    (actions['event'] != 'building')
    ]
values = [0.07, 0]
actions['ad_rev'] = np.select(conditions, values)
actions.head()

,dt,event,building_type,user_id,project_type,lvl_finished,ad_rev
0,2020-05-04 00:00:01,building,assembly_shop,55e92310-cb8e-4754-b622-597e124b03de,NaN,False,0.07
1,2020-05-04 00:00:03,building,assembly_shop,c07b1c10-f477-44dc-81dc-ec82254b1347,NaN,True,0.07
2,2020-05-04 00:00:16,building,assembly_shop,6edd42cc-e753-4ff6-a947-2107cd560710,NaN,True,0.07
3,2020-05-04 00:00:16,building,assembly_shop,92c69003-d60a-444a-827f-8cc51bf6bf4c,NaN,False,0.07
4,2020-05-04 00:00:35,building,assembly_shop,cdc6bb92-0ccb-4490-9866-ef142f09139d,NaN,True,0.07


Смотрю пропуски.

In [12]:
actions.query('project_type.isna()').head()

,dt,event,building_type,user_id,project_type,lvl_finished,ad_rev
0,2020-05-04 00:00:01,building,assembly_shop,55e92310-cb8e-4754-b622-597e124b03de,NaN,False,0.07
1,2020-05-04 00:00:03,building,assembly_shop,c07b1c10-f477-44dc-81dc-ec82254b1347,NaN,True,0.07
2,2020-05-04 00:00:16,building,assembly_shop,6edd42cc-e753-4ff6-a947-2107cd560710,NaN,True,0.07
3,2020-05-04 00:00:16,building,assembly_shop,92c69003-d60a-444a-827f-8cc51bf6bf4c,NaN,False,0.07
4,2020-05-04 00:00:35,building,assembly_shop,cdc6bb92-0ccb-4490-9866-ef142f09139d,NaN,True,0.07


In [13]:
actions.query('building_type.isna()').head()

,dt,event,building_type,user_id,project_type,lvl_finished,ad_rev
6659,2020-05-04 19:47:29,finished_stage_1,NaN,ced7b368-818f-48f6-9461-2346de0892c5,NaN,True,0.0
13134,2020-05-05 13:22:09,finished_stage_1,NaN,7ef7fc89-2779-46ea-b328-9e5035b83af5,NaN,True,0.0
15274,2020-05-05 18:54:37,finished_stage_1,NaN,70db22b3-c2f4-43bc-94ea-51c8d2904a29,NaN,True,0.0
16284,2020-05-05 21:27:29,finished_stage_1,NaN,903fc9ef-ba97-4b12-9d5c-ac8d602fbd8b,NaN,True,0.0
19650,2020-05-06 06:02:22,finished_stage_1,NaN,58e077ba-feb1-4556-a5a0-d96bd04efa39,NaN,True,0.0


In [14]:
actions.query('event == "building"')['building_type'].unique()

array(['assembly_shop', 'spaceport', 'research_center'], dtype=object)

В строительстве пропусков нет.

In [15]:
actions.query('event == "building"')['project_type'].unique()

array([nan], dtype=object)

Также при строительстве невозможно событие проект, все правильно.

In [16]:
actions.query('event == "project"')['building_type'].unique()

array([nan], dtype=object)

In [17]:
actions.query('event == "project"')['project_type'].unique()

array(['satellite_orbital_assembly'], dtype=object)

При событии проект - возможет только проект и нет типа здания - так и должно быть.

Итог предобработки: удален 1 дупликат, изменен тип дат, добавлен столбец "уровень завершен" и цена за просмотр рекламы, так же имеются пропуски, но они обусловлены механикой записи данных в игре, тк после завершения уровня - игрок ничего не строит и не исследует и дальнейшие записи не делаются.

## Провести исследовательский анализ данных
- Разбить всех на исследователей и воителей, если они завершили уровень.
- Определить время на завершение уровня для каждого юзера.
- Предложить схему монетизации на основе полученных данных.

### Разделение пользоватлей на стратегии выйгрыша

In [18]:
winners = pd.pivot_table(actions.query('lvl_finished == True'), index = 'user_id', columns = ['event'], values = 'lvl_finished', aggfunc ='nunique')
winners.columns = ['build', 'finish','project']
winners = winners[['finish', 'project']]
winners.head()

,finish,project
user_id,,
001d39dc-366c-4021-9604-6a3b9ff01e25,1.0,NaN
003ee7e8-6356-4286-a715-62f22a18d2a1,1.0,NaN
004074f8-251a-4776-922c-8348dca1a2f6,1.0,NaN
00464e5e-28ed-4be6-bf44-517ef7b3937b,1.0,NaN
0056c007-c082-43cc-bdef-afc49bca4cb9,1.0,NaN


In [19]:
conditions = \
[
    (
        (winners['finish'].transform(lambda x: x==1))& \
        (winners['project'].transform(lambda x: x==1))
    ),
    (
        (winners['finish'].transform(lambda x: x == 1))& \
        (winners['project'].transform(lambda x: x!=1))
    ),
        (winners['finish'].transform(lambda x: x!=1))
]
values = ['peace', 'war', 'none']
winners['strat'] = np.select(conditions, values)
winners.head()

,finish,project,strat
user_id,,,
001d39dc-366c-4021-9604-6a3b9ff01e25,1.0,NaN,war
003ee7e8-6356-4286-a715-62f22a18d2a1,1.0,NaN,war
004074f8-251a-4776-922c-8348dca1a2f6,1.0,NaN,war
00464e5e-28ed-4be6-bf44-517ef7b3937b,1.0,NaN,war
0056c007-c082-43cc-bdef-afc49bca4cb9,1.0,NaN,war


In [20]:
strat_count = winners['strat'].value_counts().reset_index()
strat_count = strat_count.rename(columns={'index': 'strat', 'strat': 'count'})
strat_count

,strat,count
0,war,3951
1,peace,1866


In [21]:
fig = px.bar(strat_count, x="strat", y='count', text = 'count', title="Игроки, закончившие уровень разными стратегиями", \
             labels={'count':'Количество','strat':'Стратегия'})
fig.show()

### Время прохождения уровня

In [22]:
winners['dt_min'] = actions.groupby('user_id')['dt'].min()
dt_finish = actions.groupby(['user_id','event'])['dt'].first().to_frame().reset_index()
dt_finish = dt_finish[dt_finish['event']=='finished_stage_1'][['user_id','dt']]
win = pd.merge(winners, dt_finish, on = 'user_id')
win = win.rename(columns={'dt': 'dt_finish'})
win['lvl_dt'] = win['dt_finish'] - win['dt_min']
win_short = win[['user_id','strat','lvl_dt']]
#секунды для проверки гипотезы
win_short['lvl_dt_sec'] = win_short['lvl_dt'].dt.total_seconds()
display(win.head())
display(win_short.head())

,user_id,finish,project,strat,dt_min,dt_finish,lvl_dt
0,001d39dc-366c-4021-9604-6a3b9ff01e25,1.0,NaN,war,2020-05-05 21:02:05,2020-05-12 07:40:47,6 days 10:38:42
1,003ee7e8-6356-4286-a715-62f22a18d2a1,1.0,NaN,war,2020-05-05 08:31:04,2020-05-17 21:47:49,12 days 13:16:45
2,004074f8-251a-4776-922c-8348dca1a2f6,1.0,NaN,war,2020-05-04 01:07:35,2020-05-16 19:57:40,12 days 18:50:05
3,00464e5e-28ed-4be6-bf44-517ef7b3937b,1.0,NaN,war,2020-05-04 08:35:29,2020-05-15 16:02:01,11 days 07:26:32
4,0056c007-c082-43cc-bdef-afc49bca4cb9,1.0,NaN,war,2020-05-10 11:49:47,2020-05-16 13:48:39,6 days 01:58:52


,user_id,strat,lvl_dt,lvl_dt_sec
0,001d39dc-366c-4021-9604-6a3b9ff01e25,war,6 days 10:38:42,556722.0
1,003ee7e8-6356-4286-a715-62f22a18d2a1,war,12 days 13:16:45,1084605.0
2,004074f8-251a-4776-922c-8348dca1a2f6,war,12 days 18:50:05,1104605.0
3,00464e5e-28ed-4be6-bf44-517ef7b3937b,war,11 days 07:26:32,977192.0
4,0056c007-c082-43cc-bdef-afc49bca4cb9,war,6 days 01:58:52,525532.0


Найдем общее и среднее время прохождения уровня, в зависимости от стратегии.

In [23]:
win_short.groupby('strat')['lvl_dt'].sum().reset_index()

,strat,lvl_dt
0,peace,25114 days 07:28:06
1,war,43873 days 15:55:25


Секунды понадобятся дял проверки гипотезы*

In [24]:
win_short.groupby('strat')['lvl_dt_sec'].median()

strat
peace    1125941.0
war       932497.0
Name: lvl_dt_sec, dtype: float64

А часы для удобства

In [25]:
(win_short.groupby('strat')['lvl_dt_sec'].median()/3600).round(2).reset_index()

,strat,lvl_dt_sec
0,peace,312.76
1,war,259.03


In [26]:
fig = px.bar((win_short.groupby('strat')['lvl_dt_sec'].median()/3600).round(2).reset_index(), x='strat', y='lvl_dt_sec', text = 'lvl_dt_sec', title="Среднее время прохожденяи уровня", labels={'strat':'Стратегия','lvl_dt_sec':'Время, ч'})
fig.show()

Время, которое пользватели тратят на прохождения уровня с помощью стратегии войны быстрее, чем с помощью стратегии исследования.

Это суждение проверится в блоке гипотез.

### Влияние событий на совершение целевого события
- Определить количество построенных зданий каждым юзером и сопоставить с условием победы.
- Найти самые частые постройки.
- Расчитать доход с показов рекламы после построек зданий, считая что один доход за показ составляет 0.07 у.е.

### Выделение датасета по строительству

In [27]:
actions_build = actions.join(winners, on='user_id', how='left')
actions_build['strat'] = actions_build['strat'].fillna('no_strat')
actions_build.head()

,dt,event,building_type,user_id,project_type,lvl_finished,ad_rev,finish,project,strat,dt_min
0,2020-05-04 00:00:01,building,assembly_shop,55e92310-cb8e-4754-b622-597e124b03de,NaN,False,0.07,NaN,NaN,no_strat,NaT
1,2020-05-04 00:00:03,building,assembly_shop,c07b1c10-f477-44dc-81dc-ec82254b1347,NaN,True,0.07,1.0,1.0,peace,2020-05-04 00:00:03
2,2020-05-04 00:00:16,building,assembly_shop,6edd42cc-e753-4ff6-a947-2107cd560710,NaN,True,0.07,1.0,NaN,war,2020-05-04 00:00:16
3,2020-05-04 00:00:16,building,assembly_shop,92c69003-d60a-444a-827f-8cc51bf6bf4c,NaN,False,0.07,NaN,NaN,no_strat,NaT
4,2020-05-04 00:00:35,building,assembly_shop,cdc6bb92-0ccb-4490-9866-ef142f09139d,NaN,True,0.07,1.0,NaN,war,2020-05-04 00:00:35


In [28]:
build_by_dt = actions_build.query('event == "building"').groupby(['dt'])['building_type'].count().reset_index()
build_by_dt['dt'] = pd.to_datetime(build_by_dt['dt']).dt.date
build_by_dt = build_by_dt.groupby('dt').count().reset_index()
build_by_dt.head()

,dt,building_type
0,2020-05-04,7826
1,2020-05-05,8569
2,2020-05-06,9396
3,2020-05-07,10206
4,2020-05-08,11256


In [29]:
fig = px.bar(build_by_dt, x='dt', y='building_type', title="Построенные здания за период", labels={'dt':'Дата','building_type':'Кол-во зданий'})
fig.show()

In [30]:
build_by_strat = actions_build.query('building_type.isna() == False').groupby(['user_id','strat', 'building_type', 'ad_rev'])['event'].count().reset_index()
build_by_strat = build_by_strat.rename(columns={'event': 'qty'})
build_by_strat['ad_rev'] = build_by_strat['ad_rev']*build_by_strat['qty']
build_by_strat.head()

,user_id,strat,building_type,ad_rev,qty
0,0001f83c-c6ac-4621-b7f0-8a28b283ac30,no_strat,assembly_shop,0.42,6
1,0001f83c-c6ac-4621-b7f0-8a28b283ac30,no_strat,research_center,0.07,1
2,0001f83c-c6ac-4621-b7f0-8a28b283ac30,no_strat,spaceport,0.42,6
3,00151b4f-ba38-44a8-a650-d7cf130a0105,no_strat,assembly_shop,0.28,4
4,00151b4f-ba38-44a8-a650-d7cf130a0105,no_strat,spaceport,0.35,5


Количество зданий и доход на пользователя.

In [31]:
build_by_user = build_by_strat.groupby('user_id')['qty'].sum().reset_index()
build_by_user['revenue'] = build_by_user['qty']*0.07
build_by_user.head()

,user_id,qty,revenue
0,0001f83c-c6ac-4621-b7f0-8a28b283ac30,13,0.91
1,00151b4f-ba38-44a8-a650-d7cf130a0105,9,0.63
2,001aaea6-3d14-43f1-8ca8-7f48820f17aa,4,0.28
3,001d39dc-366c-4021-9604-6a3b9ff01e25,8,0.56
4,002f508f-67b6-479f-814b-b05f00d4e995,12,0.84


In [32]:
revenue__by_user_sum = build_by_user['revenue'].sum()
revenue__by_user_sum

8956.92

In [33]:
build_by_user_mean = build_by_user.mean()
build_by_user_mean

qty        9.425162
revenue    0.659761
dtype: float64

### Количество различных построеных зданий в зависимости от стратегии

In [34]:
build_by_user_strat = build_by_strat.groupby(['strat','building_type'])['qty'].sum().sort_values().reset_index()
build_by_user_strat

,strat,building_type,qty
0,peace,research_center,3697
1,war,research_center,3755
2,no_strat,research_center,6685
3,peace,assembly_shop,9116
4,peace,spaceport,10788
5,war,assembly_shop,16298
6,war,spaceport,17132
7,no_strat,assembly_shop,29080
8,no_strat,spaceport,31405


In [35]:
fig = px.bar(build_by_user_strat, x="strat", y='qty', text = 'qty', color = 'building_type',barmode='group', title="Количество различных построеных зданий по стратегии", labels={'qty':'Количество','strat':'Стратегия'})
fig.show()

In [36]:
type_count = build_by_strat.groupby('building_type').sum().sort_values(by = 'qty').reset_index()
type_count

,building_type,ad_rev,qty
0,research_center,989.59,14137
1,assembly_shop,3814.58,54494
2,spaceport,4152.75,59325


In [37]:
fig = px.bar(type_count, x="building_type", y='qty', text = 'qty', title="Количество построеных зданий по типам", labels={'qty':'Количество','building_type':'Тип здания'})
fig.show()

### Средние значения прибыли и количества зданий в зависимости от стратегии

In [38]:
build_by_strat.groupby('building_type').mean().reset_index()

,building_type,ad_rev,qty
0,assembly_shop,0.280980,4.013995
1,research_center,0.129004,1.842915
2,spaceport,0.313865,4.483788


Найдем количество зданий, построенных игроками, в зависимости от стратегии.

In [39]:
build_by_strat.groupby('strat')['qty'].sum().sort_values().reset_index()

,strat,qty
0,peace,23601
1,war,37185
2,no_strat,67170


Считаю количество зданий на пользователя, построенных, в зависимости от стратегии.

In [40]:
user_builds = pd.pivot_table(build_by_strat, index = 'user_id', columns = 'strat', values = 'qty', aggfunc = 'sum').reset_index()
user_builds.head()

strat,user_id,no_strat,peace,war
0,0001f83c-c6ac-4621-b7f0-8a28b283ac30,13.0,NaN,NaN
1,00151b4f-ba38-44a8-a650-d7cf130a0105,9.0,NaN,NaN
2,001aaea6-3d14-43f1-8ca8-7f48820f17aa,4.0,NaN,NaN
3,001d39dc-366c-4021-9604-6a3b9ff01e25,NaN,NaN,8.0
4,002f508f-67b6-479f-814b-b05f00d4e995,12.0,NaN,NaN


In [41]:
df = user_builds.mean().round(2).sort_values().reset_index()
df

,strat,0
0,no_strat,8.66
1,war,9.41
2,peace,12.65


In [42]:
fig = px.bar(df, x="strat", y=0, title="Количество построеных зданий по типам", text = 0, labels={'strat':'Стратегия', '0':'Количество'})
fig.show()

**Итого**: наблюдаем такую картину: пве-игроки в среднем строят больше всех, далее идут пвп-игроки, и меньше всех строят игроки, которые не прошли уровень.

### Прибыль с рекламы

Посчитаем доход с рекламы.

In [43]:
total_costs = costs['cost'].sum()
total_revenue = build_by_strat['qty'].sum()*0.07
dif = total_revenue - total_costs
print('Всего потрачено на рекламу:', total_costs.round(2), 'у.е')
print('Всего заработано на рекламе:', total_revenue.round(2), 'у.е')
print('Прибыль с рекламы:', dif.round(2), 'у.е')

Всего потрачено на рекламу: 7603.58 у.е
Всего заработано на рекламе: 8956.92 у.е
Прибыль с рекламы: 1353.34 у.е


In [44]:
x = ['Costs','Revenue']
y=[-total_costs.round(2),total_revenue]
fig = px.bar(y=y, x=x, text = y,
            title="Costs/Revenue", color = y)
fig.update_traces(textfont_size=12, textangle=0, textposition="inside", cliponaxis=False)
fig.show()

Делаем вывод, что данная модель монетизации уже работает - реклама окупается.

Можно сделать вывод, что пользователи намного охотнее идут по пути войны, это видно по количеству завершенных уровней этой стратегией - 3951 против 1866, которое подтверждается постройками assembly_shop и spaceport, которые помогают нарастить военную мощь, а research_center строят гораздо реже, и выигрывают стратегией исследования очевидно реже.
Также в целом пвп-игроки строят больше чем пве, однако не прошедшие уровень - строят еще больше.

Заказчик на данный момент выходит в + (1353у.е), однако задача - уменьшить количество рекламы и выйти в ноль.

### Расчет среднего расхода на пользователя

In [45]:
total_users = actions['user_id'].nunique()
print('Всего пользователей:', total_users)

Всего пользователей: 13576


In [46]:
cost_by_user = (total_costs/total_users).round(2)
print('Расход на пользователя',cost_by_user, 'у.е.')

Расход на пользователя 0.56 у.е.


In [47]:
build_to_zero = (cost_by_user/0.07).round(2)
print('Количество зданий, которое должен построить пользователь, чтобы выйти в ноль:', build_to_zero)

Количество зданий, которое должен построить пользователь, чтобы выйти в ноль: 8.0


In [48]:
build_to_plus = float(build_by_user['qty'].mean().round(2))
print('Суммарное среднее количество зданий, которое построит один пользователь:', build_to_plus)

Суммарное среднее количество зданий, которое построит один пользователь: 9.43


In [49]:
data = [['old',total_users,cost_by_user, build_to_plus, 0.07],['new',total_users,cost_by_user, build_to_zero,0.07]]
money_df = pd.DataFrame(data, columns=['model', 'total_users','cost_per_user','build_by_user','ad_rev'])
money_df['costs'] = -money_df['cost_per_user']*money_df['total_users']
money_df['revenue'] = money_df['build_by_user']*money_df['ad_rev']*money_df['total_users']
money_df['diff'] = money_df['revenue']+money_df['costs']
money_df['revenue'] = money_df['revenue'].round(2)
money_df['diff'] = money_df['diff'].round(2)
money_df

,model,total_users,cost_per_user,build_by_user,ad_rev,costs,revenue,diff
0,old,13576,0.56,9.43,0.07,-7602.56,8961.52,1358.96
1,new,13576,0.56,8.00,0.07,-7602.56,7602.56,0.00


In [50]:
y_1 = money_df['costs']
y_2 = money_df['revenue']
x = money_df['model']

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=y_1, name = 'costs', text = y_1))
fig.add_trace(go.Bar(x=x, y=y_2, name = 'revenue', text = y_2))
fig.update_layout(barmode='relative', title_text='Old/New model')
fig.show()

**Вывод**

Исходя из расчетов, можно сказать следующее: чтобы выходить в ноль - каждый пользователь должен построить минимум 8 зданий, в данный момент каждый в среднем строит 9.4 зданий, тем самым выходит в плюс.

Если мы хотим, чтобы пользователя не напрягала реклама и при этом нужно выйти в ноль - предлагаю схему ограничения показа рекламы после постройки 8ми зданий на игрока, при учете что расходы на рекламу остаются теже и механика уровня остается без изменений, данное предложение касается только 1го уровня в таком виде, какой он сейчас есть.

In [51]:
build_by_user.head()

,user_id,qty,revenue
0,0001f83c-c6ac-4621-b7f0-8a28b283ac30,13,0.91
1,00151b4f-ba38-44a8-a650-d7cf130a0105,9,0.63
2,001aaea6-3d14-43f1-8ca8-7f48820f17aa,4,0.28
3,001d39dc-366c-4021-9604-6a3b9ff01e25,8,0.56
4,002f508f-67b6-479f-814b-b05f00d4e995,12,0.84


Добавлю столбы обозначающие, что реклама показывается на 2ю и 3ю постройку и посчитаю прибыль.

In [52]:
build_by_user['qty_1'] = build_by_user['qty']-1
build_by_user['rev_1'] = build_by_user['qty_1']*0.07
build_by_user['qty_2'] = build_by_user['qty']-2
build_by_user['rev_2'] = build_by_user['qty_2']*0.07
display(build_by_user.head())
display(build_by_user[['rev_1','rev_2']].sum())

,user_id,qty,revenue,qty_1,rev_1,qty_2,rev_2
0,0001f83c-c6ac-4621-b7f0-8a28b283ac30,13,0.91,12,0.84,11,0.77
1,00151b4f-ba38-44a8-a650-d7cf130a0105,9,0.63,8,0.56,7,0.49
2,001aaea6-3d14-43f1-8ca8-7f48820f17aa,4,0.28,3,0.21,2,0.14
3,001d39dc-366c-4021-9604-6a3b9ff01e25,8,0.56,7,0.49,6,0.42
4,002f508f-67b6-479f-814b-b05f00d4e995,12,0.84,11,0.77,10,0.70


rev_1    8006.60
rev_2    7056.28
dtype: float64

In [53]:
t_c = -(total_costs.round(2))
y_1 = [t_c, t_c, t_c]
y_2 = [build_by_user['revenue'].sum(),build_by_user['rev_1'].sum(),build_by_user['rev_2'].sum().round(2)]
x = ['old','new_1','new_2']

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=[-t_c, -t_c, -t_c], name='rev+',
                         line = dict(color='gold', width=4, dash='dash')))
fig.add_trace(go.Bar(x=x, y=y_1, name = 'costs', text = y_1))
fig.add_trace(go.Bar(x=x, y=y_2, name = 'revenue', text = y_2,textposition=['auto','outside','inside']))
fig.update_layout(barmode='relative', title_text='Old/New model')

fig.show()

Итоговая выручка.

In [54]:
print('Выручка при показе рекламы со 2го здания:',(build_by_user['rev_1'].sum()-total_costs).round(2), 'у.е.')
print('Выручка при показе рекламы со 3го здания:',(build_by_user['rev_2'].sum()-total_costs).round(2), 'у.е.')

Выручка при показе рекламы со 2го здания: 403.02 у.е.
Выручка при показе рекламы со 3го здания: -547.3 у.е.


**Вывод:** если показывать рекламу при постройке 2го здания - реклама все еще окупается и остается в небольшом плюсе, если показывать с 3й пострйокb, то выручка уходит в минус.

Следовательно можно предложить такую модель - показывать рекламу пользователю начиная с постройки 2го здания, тем самым пользователи будут смотреть меньше рекламы, а реклама будет все равно оккупаться.

## Проверьте статистические гипотезы
- Проверить гипотезу различия времени прохождения уровня между пользователями, которые заканчивают уровень через реализацию проекта, и пользователями, которые заканчивают уровень победой над другим игроком.
- Сформулировать и проверить статистическую гипотезу относительно представленных данных.

Гипотеза: Предположим, что пользователи проходят уровень одинаково быстро, независимо от стратегии.

H0 - Статистической значимости в различии времени прохождения уровня между стратегиями нет.

H1 - Статистическая значимость в различии времени прохождения уровня между стратегиями есть.

In [55]:
win_short.head()

,user_id,strat,lvl_dt,lvl_dt_sec
0,001d39dc-366c-4021-9604-6a3b9ff01e25,war,6 days 10:38:42,556722.0
1,003ee7e8-6356-4286-a715-62f22a18d2a1,war,12 days 13:16:45,1084605.0
2,004074f8-251a-4776-922c-8348dca1a2f6,war,12 days 18:50:05,1104605.0
3,00464e5e-28ed-4be6-bf44-517ef7b3937b,war,11 days 07:26:32,977192.0
4,0056c007-c082-43cc-bdef-afc49bca4cb9,war,6 days 01:58:52,525532.0


In [56]:
df = win_short
samp1 = df[df['strat']=='war']['lvl_dt_sec']
samp2 = df[df['strat']=='peace']['lvl_dt_sec']

Проверим на нормальность данные:

In [57]:
alpha = 0.05
results = st.shapiro(samp1)
p_value = results[1]
print('p-значение: ', p_value)
if p_value < alpha:
    print('Распределение не нормально')
else:
    print('Распределение нормально') 

p-значение:  1.7430939153124487e-21
Распределение не нормально


In [58]:
alpha = 0.05
results = st.shapiro(samp2)
p_value = results[1]
print('p-значение: ', p_value)
if p_value < alpha:
    print('Распределение не нормально')
else:
    print('Распределение нормально') 

p-значение:  3.415951876345406e-20
Распределение не нормально


Данные не нормальные, значит используем тест Манна-Уитни

In [59]:
alpha = 0.05
results = st.mannwhitneyu(samp1, samp2)
print('p-значение: ', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 

p-значение:  1.1228044721644527e-106
Отвергаем нулевую гипотезу: разница статистически значима


Видим, что различия есть, причем очень весомые, значит время прохождения уровня с большой вероятностью зависит от выбранной стратегии.

### Зависимость прибыли в зависимости от различных построенных зданий.

Гипотеза: Предположим, что прибыль одинакова для каждого типа зданий.

H0 - Статистической значимости в различии прибыли в зависимости от типа зданий нет.

H1 - Статистической значимость в различии прибыли в зависимости от типа зданий есть.

In [60]:
build_by_strat.head()

,user_id,strat,building_type,ad_rev,qty
0,0001f83c-c6ac-4621-b7f0-8a28b283ac30,no_strat,assembly_shop,0.42,6
1,0001f83c-c6ac-4621-b7f0-8a28b283ac30,no_strat,research_center,0.07,1
2,0001f83c-c6ac-4621-b7f0-8a28b283ac30,no_strat,spaceport,0.42,6
3,00151b4f-ba38-44a8-a650-d7cf130a0105,no_strat,assembly_shop,0.28,4
4,00151b4f-ba38-44a8-a650-d7cf130a0105,no_strat,spaceport,0.35,5


In [61]:
df = build_by_strat
s1 = df[df['building_type']=='assembly_shop']['ad_rev']
s2 = df[df['building_type']=='research_center']['ad_rev']
s3 = df[df['building_type']=='spaceport']['ad_rev']

Проверим на нормальность

In [62]:
alpha = 0.05
results = st.shapiro(s1)
p_value = results[1]
print('p-значение: ', p_value)
if p_value < alpha:
    print('Распределение не нормально')
else:
    print('Распределение нормально') 

p-значение:  0.0
Распределение не нормально


In [63]:
alpha = 0.05
results = st.shapiro(s2)
p_value = results[1]
print('p-значение: ', p_value)
if p_value < alpha:
    print('Распределение не нормально')
else:
    print('Распределение нормально') 

p-значение:  0.0
Распределение не нормально


In [64]:
alpha = 0.05
results = st.shapiro(s3)
p_value = results[1]
print('p-значение: ', p_value)
if p_value < alpha:
    print('Распределение не нормально')
else:
    print('Распределение нормально') 

p-значение:  0.0
Распределение не нормально


Данные не нормальны, используем тест Манна-Уитни

In [65]:
#используем поправку Бонферрони для 3х стравнений
alpha = 0.05/3

Сравнение assembly_shop и research_center:

In [66]:
results = st.mannwhitneyu(s1, s2)
print('p-значение: ', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 

p-значение:  0.0
Отвергаем нулевую гипотезу: разница статистически значима


Сравнение assembly_shop и spaceport:

In [67]:
results = st.mannwhitneyu(s1, s3)
print('p-значение: ', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 

p-значение:  7.710933489020153e-46
Отвергаем нулевую гипотезу: разница статистически значима


Сравнение research_center и spaceport:

In [68]:
results = st.mannwhitneyu(s2, s3, True)
print('p-значение: ', results.pvalue)
if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 

p-значение:  0.0
Отвергаем нулевую гипотезу: разница статистически значима


Как видно из тестов при всех стравнениях прибыль скорее всего зависит от типа построенного здания.

## Вывод

В ходе исследования была проведена предобработка данных - устранение дупликатов, изменение датасета, объяснение пропусков.
Также был првоеден исследовательский анализ, в ходе которого было выявлено, что:
- Игроки в зависимости от стратегии игры строят разное количество зданий: 8.7 - игроки, оставшиеся на уровне, 12.6 - пве игроки, 9.4 - пвп игроки.
- Игроки проходят уровень с разной скоростью, в зависимости от стратегии, в среднем: пве - 312 часов на уровне, пвп - 259 часов.
- Предложенная маркетологами модель монетизации работает - реклама окупается, но показывается слишком много рекламы, т.к в среднем игроки строят 9.4 зданий, а реклама окупается уже на 8м. Соответсвенное далее рекалма показывается уже в плюс.

В итоге была предложена новая модель монетизации: показывать рекламу пользователю начиная с постройки 2го здания, тем самым пользователи будут смотреть меньше рекламы, которая в свою очередь будет все равно оккупаться.